# spiral_wasm

In [ ]:
open rust.rust_operators

## spiral_wasm

### main

In [ ]:
// fn print_usd(result: near_workspaces::result::ExecutionFinalResult) {
//     // const GAS_PRICE_IN_NEAR: f64 = 0.0001;
//     const NEAR_PRICE_IN_USD: f64 = 6.68;
// 
//     fn gas_to_usd(gas: u64) -> f64 {
//         (gas as f64) / 1e16 * NEAR_PRICE_IN_USD
//     }
// 
//     fn tokens_to_usd(tokens: u128) -> f64 {
//         (tokens as f64) / 1e24 * NEAR_PRICE_IN_USD
//     }
// 
//     println!(
//         "gas_to_usd(result.total_gas_burnt.as_gas()): {:.6} ({:#?})",
//         gas_to_usd(result.total_gas_burnt.as_gas()), result.total_gas_burnt.as_gas()
//     );
//     result.outcomes().iter().for_each(|outcome| {
//         println!("outcome (success: {:#?}):", outcome.is_success());
//         println!(
//             "  gas_to_usd(outcome.gas_burnt.as_gas()): {:.6} ({:#?})",
//             gas_to_usd(outcome.gas_burnt.as_gas()), outcome.gas_burnt.as_gas()
//         );
//         println!(
//             "  tokens_to_usd(outcome.tokens_burnt.as_yoctonear()): {:.6} ({:#?})",
//             tokens_to_usd(outcome.tokens_burnt.as_yoctonear()), outcome.tokens_burnt.as_yoctonear()
//         );
//     });
// }
// 
// #[tokio::main]
// async fn main() -> anyhow::Result<()> {
//     let wasm_path = std::env::var("WASM_PATH").unwrap();
// 
//     let worker = near_workspaces::sandbox().await?;
//     let wasm = std::fs::read(wasm_path)?;
//     let contract = worker.dev_deploy(&wasm).await?;
// 
//     let result = contract.call("state_main").transact().await?;
// 
//     let logs = result.logs();
//     logs.iter().for_each(|log| {
//         println!("{}", log.trim());
//     });
// 
//     println!("\n\n state_main transact result: {result:#?}");
// 
//     println!("\n\n print_usd:");
//     print_usd(result.clone());
// 
//     let result2 = result.clone().into_result();
//     println!("\n\n result.into_result(): {result2:#?}");
// 
//     let receipt_failures = result.receipt_failures();
//     println!("\n\n receipt_failures: {receipt_failures:#?}");
// 
//     let receipt_outcomes = result.receipt_outcomes();
//     println!("\n\n receipt_outcomes: {receipt_outcomes:#?}");
// 
//     let json: Result<String, near_workspaces::error::Error> = result.clone().json();
//     println!("\n\n json: {json:#?}");
// 
//     let borsh: Result<String, near_workspaces::error::Error> = result.clone().borsh();
//     println!("\n\n borsh: {borsh:#?}");
// 
//     println!("\n\n worker: {:#?}", worker);
//     println!("\n\n contract: {:#?}", contract);
// 
//     if receipt_failures.len() == 0 {
//         Ok(())
//     } else {
//         // let err = receipt_failures
//         //     .iter()
//         //     .map(|outcome| {
//         //         let result = <near_workspaces::result::ExecutionOutcome as Clone>::clone(outcome)
//         //             .into_result();
//         //         format!("outcome.into_result(): {:#?}", result)
//         //     })
//         //     .collect::<Vec<_>>();
//         // println!("\n\n err: {:#?}", err);
//         // let err = format!("{:#?}", err);
//         // let err = anyhow::anyhow!(err);
//         let err = anyhow::anyhow!("<error>");
//         Err(err)
//     }
// }

In [ ]:
///! _

inl main () =
    !\($'"} //"') : ()

    // !\($'"#[near_sdk::near_bindgen] //"') : ()

    // !\($'"#[derive( //"') : ()
    // !\($'"  near_sdk::PanicOnDefault, //"') : ()
    // !\($'"  borsh::BorshDeserialize, //"') : ()
    // !\($'"  borsh::BorshSerialize, //"') : ()
    // !\($'")] //"') : ()

    // !\($'"pub struct State { //"') : ()
    // !\($'"} //"') : ()


    // !\($'"#[near_sdk::near_bindgen] //"') : ()
    // !\($'"impl State { //"') : ()

    // !\($'"#[init] //"') : ()
    // !\($'"pub fn new() -> Self { //"') : ()
    // !\($'"  Self { //"') : ()
    // !\($'"  } //"') : ()
    // !\($'"} //"') : ()

    // !\($'"#[result_serializer(borsh)] //"') : ()
    // !\($'"pub fn generate_cid_borsh( //"') : ()
    // !\($'"  &self, //"') : ()
    // !\($'"  #[serializer(borsh)] content: Vec<u8>, //"') : ()
    // !\($'") -> String { //"') : ()
    // !\($'"\\\"\\\".to_string() //"') : ()
    // !\($'"} //"') : ()

    // !\($'"pub fn claim_alias( //"') : ()
    // !\($'"  &mut self, //"') : ()
    // !\($'"  alias: String, //"') : ()
    // !\($'") { //"') : ()
    // !\($'"} //"') : ()

    // !\($'"pub fn get_account_info( //"') : ()
    // !\($'"  &self, //"') : ()
    // !\($'"  account_id: near_sdk::AccountId, //"') : ()
    // !\($'") -> Option<(String, (u64, u32))> { //"') : ()
    // !\($'"None //"') : ()
    // !\($'"} //"') : ()

    // !\($'"pub fn get_alias_map( //"') : ()
    // !\($'"  &self, //"') : ()
    // !\($'"  alias: String, //"') : ()
    // !\($'") -> Option<std::collections::HashMap<near_sdk::AccountId, (u64, u32)>> { //"') : ()
    // !\($'"None //"') : ()
    // !\($'"} //"') : ()

    // !\($'"#[result_serializer(borsh)] //"') : ()
    // !\($'"pub fn get_alias_map_borsh( //"') : ()
    // !\($'"  &self, //"') : ()
    // !\($'"  #[serializer(borsh)] alias: String, //"') : ()
    // !\($'") -> Option<std::collections::HashMap<near_sdk::AccountId, (u64, u32)>> { //"') : ()
    // !\($'"None //"') : ()
    // !\($'"} //"') : ()

    // !\($'"} //"') : ()

    !\($'"#[tokio::main] //"') : ()
    !\($'"async fn main() -> anyhow::Result<()> { //"') : ()
    !\($'"  let wasm_path = std::env::var(\\\"WASM_PATH\\\").unwrap(); //"') : ()

    !\($'"  let worker = near_workspaces::sandbox().await?; //"') : ()
    !\($'"  let wasm = std::fs::read(wasm_path)?; //"') : ()
    !\($'"  let contract = worker.dev_deploy(&wasm).await?; //"') : ()

    !\($'"  let result = contract.call(\\\"state_main\\\").transact().await?; //"') : ()

    !\($'"  let logs = result.logs(); //"') : ()
    !\($'"  logs.iter().for_each(|log| { //"') : ()
    !\($'"    println\!(\\\"{}\\\", log.trim()); //"') : ()
    !\($'"  }); //"') : ()

    !\($'"  println\!(\\\"\\\\n\\\\n state_main transact result: {result:#?}\\\"); //"') : ()

    !\($'"  println\!(\\\"\\\\n\\\\n print_usd:\\\"); //"') : ()
    !\($'"  print_usd(result.clone()); //"') : ()

    !\($'"  let result2 = result.clone().into_result(); //"') : ()
    !\($'"  println\!(\\\"\\\\n\\\\n result.into_result(): {result2:#?}\\\"); //"') : ()

    !\($'"  let receipt_failures = result.receipt_failures(); //"') : ()
    !\($'"  println\!(\\\"\\\\n\\\\n receipt_failures: {receipt_failures:#?}\\\"); //"') : ()

    !\($'"  let receipt_outcomes = result.receipt_outcomes(); //"') : ()
    !\($'"  println\!(\\\"\\\\n\\\\n receipt_outcomes: {receipt_outcomes:#?}\\\"); //"') : ()

    !\($'"  let json: Result<String, near_workspaces::error::Error> = result.clone().json(); //"') : ()
    !\($'"  println\!(\\\"\\\\n\\\\n json: {json:#?}\\\"); //"') : ()

    !\($'"  let borsh: Result<String, near_workspaces::error::Error> = result.clone().borsh(); //"') : ()
    !\($'"  println\!(\\\"\\\\n\\\\n borsh: {borsh:#?}\\\"); //"') : ()

    !\($'"  println\!(\\\"\\\\n\\\\n worker: {:#?}\\\", worker); //"') : ()
    !\($'"  println\!(\\\"\\\\n\\\\n contract: {:#?}\\\", contract); //"') : ()

    !\($'"  if receipt_failures.len() == 0 { //"') : ()
    !\($'"    Ok(()) //"') : ()
    !\($'"  } else { //"') : ()
    !\($'"    let err = anyhow::anyhow\!(\\\"<error>\\\"); //"') : ()
    !\($'"    Err(err) //"') : ()
    !\($'"  } //"') : ()

    // !\($'"fn _main() { //"') : ()

inl main () =
    $'!main |> ignore' : ()